In [1]:
import uproot
import pandas as pd

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
smote=SMOTE()
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier()
from sklearn.metrics import confusion_matrix,accuracy_score

### Approach - 1

In [5]:
data_01=uproot.open('./DataGenerated/01_Downsteam_Ghost.root:Particle_Data')
data_01=data_01.arrays(library='pd')

In [7]:
features=['track_chi2PerDoF','track_nLHCbIDs','track_p','track_position_x',
           'track_position_y','track_position_r','track_pt','track_tx','track_ty','track_pseudoRapidity']


tracks=['particle_hasScifi','particle_hasUT','particle_hasVelo','particle_isDown','particle_isDown_noVelo','particle_isLong',
           'particle_isLong_andUT','partiole_isElectron']

In [16]:
data_01=data_01[features+['Downstream/Ghost']]

In [18]:
x=data_01.drop('Downstream/Ghost',axis=1)
y=data_01['Downstream/Ghost']

In [24]:
x,y=smote.fit_resample(x,y)

In [26]:
x_train,x_test,y_train,y_test=train_test_split(x,y,stratify=y,random_state=56,test_size=0.3)

In [29]:
dt.fit(x_train,y_train)
y_pred=dt.predict(x_test)
print(accuracy_score(y_test,y_pred))
confusion_matrix(y_test,y_pred)

DecisionTreeClassifier()

### Approach - 2

In [34]:
ds_data=uproot.open('./DataGenerated/02_Flagged.root:Particle_Data')
ds_data=ds_data.arrays(library='pd')

gs_data=uproot.open('./DataGenerated/02_Ghost.root:Particle_Data')
gs_data=gs_data.arrays(library='pd')

In [41]:
ds_data=ds_data[features+['particle_isDown_noVelo']]
gs_data=gs_data[features]

In [42]:
x=ds_data.drop('particle_isDown_noVelo',axis=1)
y=ds_data['particle_isDown_noVelo']

In [44]:
x,y=smote.fit_resample(x,y)

In [45]:
x_train,x_test,y_train,y_test=train_test_split(x,y,stratify=y,random_state=56,test_size=0.3)

In [46]:
dt.fit(x_train,y_train)
y_pred=dt.predict(x_test)
print(accuracy_score(y_test,y_pred))
confusion_matrix(y_test,y_pred)

0.8108675821143981


array([[1067900,  277707],
       [ 231289, 1114319]])

In [47]:
#Validating the model on Ghost:

In [48]:
ghost_predict=dt.predict(gs_data)

In [49]:
gs_data['Label']=ghost_predict

/tmp/ipykernel_22512/1238808484.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gs_data['Label']=ghost_predict


In [50]:

gs_data.head()

,track_chi2PerDoF,track_nLHCbIDs,track_p,track_position_x,track_position_y,track_position_r,track_pt,track_tx,track_ty,track_pseudoRapidity,Label
0,0.168421,11.0,11909.759099,-630.108887,94.554832,637.163892,1768.037611,-0.149710,0.011029,2.595079,False
1,0.298741,11.0,23838.467047,-185.017532,71.173920,198.235249,1378.792022,-0.057027,0.010223,3.542403,False
2,0.413166,12.0,7602.727968,399.872742,91.567459,410.222877,1175.531851,0.156025,0.012208,2.553902,False
3,0.131244,9.0,12594.646453,-588.363281,205.212540,623.124014,414.296582,-0.014740,0.029427,4.107322,False
4,0.136803,10.0,7422.665673,-383.692749,7.793806,383.771897,1139.669388,-0.155381,0.000416,2.561001,False


In [52]:
gs_data['Label'].value_counts(normalize=True)

False    0.713683
True     0.286317
Name: Label, dtype: float64